In [1]:
# Import Libraries and Dependencies
import pandas as pd

### 1. Combine and Clean the Data
#### Import CSVs

In [2]:
# Read the CSV files into DataFrames.
as2020_df = pd.read_csv('Resources/athletic_sales_2020.csv')
as2021_df = pd.read_csv('Resources/athletic_sales_2021.csv')

In [3]:
# Display the 2020 sales DataFrame
columns_2020 = as2020_df.columns.values.tolist()
as2020_df.head(3)

,retailer,retailer_id,invoice_date,region,state,city,product,price_per_unit,units_sold,total_sales,operating_profit,sales_method
0,Foot Locker,1185732,1/1/20,Northeast,New York,New York,Men's Street Footwear,50,1200,600000,300000.00,In-store
1,Foot Locker,1185732,1/1/20,Northeast,Pennsylvania,Philadelphia,Women's Apparel,68,83,5644,2426.92,Online
2,Foot Locker,1185732,1/1/20,Northeast,Pennsylvania,Philadelphia,Women's Apparel,75,275,206250,61875.00,Outlet


In [4]:
# Display the 2021 sales DataFrame
columns_2021 = as2021_df.columns.values.tolist()
as2021_df.head(3)

,retailer,retailer_id,invoice_date,region,state,city,product,price_per_unit,units_sold,total_sales,operating_profit,sales_method
0,West Gear,1128299,1/1/21,West,California,San Francisco,Men's Athletic Footwear,65,750,487500,121875.00,Outlet
1,West Gear,1128299,1/1/21,West,California,San Francisco,Men's Athletic Footwear,51,233,11883,3208.41,Outlet
2,Kohl's,1189833,1/1/21,Midwest,Montana,Billings,Men's Apparel,50,275,137500,82500.00,Outlet


#### Check the data types of each DataFrame

In [5]:
# Check the 2020 sales data types.
dtypes_2020 = as2020_df.dtypes.values.tolist()
dtypes_2020

[dtype('O'),
 dtype('int64'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('float64'),
 dtype('O')]

In [6]:
# Check the 2021 sales data types.
dtypes_2021 = as2020_df.dtypes.values.tolist()
dtypes_2021

[dtype('O'),
 dtype('int64'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('float64'),
 dtype('O')]

#### This is added. I never trust myself to manually check things like this. This reflects one way to do a basic programatic check

In [7]:
# my old eyes and memory aren't really good at comparing columns and types so I wrote this to do it for me
if columns_2020 != columns_2021:
    raise ValueError("The two data frames are not compatible. Please verify the columns.")
elif dtypes_2020 != dtypes_2021:
    raise ValueError("The two data frames are not compatible. Please verify the data types.")
else:
    print('The as2020_df and as2021_df data frames have matching columns and data types.')

The as2020_df and as2021_df data frames have matching columns and data types.


#### Combine the sales data by rows.

In [8]:
# Combine the 2020 and 2021 sales DataFrames on the rows and reset the index.
as_years_df = pd.concat([as2020_df,as2021_df],axis=0, join='inner').reset_index(drop=True)
print (f"{as_years_df[['retailer','invoice_date']].head(3)}\n{as_years_df[['retailer','invoice_date']].tail(3).to_string(header=False)}")

      retailer invoice_date
0  Foot Locker       1/1/20
1  Foot Locker       1/1/20
2  Foot Locker       1/1/20
9640       Amazon  12/31/21
9641       Amazon  12/31/21
9642  Foot Locker  12/31/21


In [9]:
''' Check if any values are null.
print (f'any is null {as_years_df.isnull().any().any()}')
print (f'any is na {as_years_df.isna().any().any()}')
print (f'any is not na {as_years_df.notna().any().any()}')
'''
if as_years_df.notna().any().any():
    print ("The dataframe has no null entries. Please continue")
else:
    print ('One or more entries is null. Please review and clean data before continuing')



The dataframe has no null entries. Please continue


In [10]:
# Check the data type of each column
# here i want to use the same check as i did earlier. 
dtype_as_years =as_years_df.dtypes.values.tolist()
if dtype_as_years != dtypes_2021:
    raise ValueError("Something is wrong with the data types in the new table")
else:
    print('The data types of the new dataframe are the same as the as2020_df and as2021_af')

The data types of the new dataframe are the same as the as2020_df and as2021_af


In [11]:
# Convert the "invoice_date" to a datetime datatype 
as_years_df['invoice_date'] = pd.to_datetime(as_years_df['invoice_date'],  format='mixed')
print (as_years_df[['retailer','invoice_date']].head(3).to_string(index=False))

   retailer invoice_date
Foot Locker   2020-01-01
Foot Locker   2020-01-01
Foot Locker   2020-01-01


In [12]:
# Confirm that the "invoice_date" data type has been changed.
# I wanted to verify that the dtype was a datetime type. 
if as_years_df['invoice_date'].dtypes != "datetime64[ns]":
    raise ValueError("Something is wrong with the data type in 'invoice_date")
else:
    print(f"The 'invoice_date' data type {as_years_df['invoice_date'].dtypes} is datetime type")

The 'invoice_date' data type datetime64[ns] is datetime type


### 2. Determine which Region Sold the Most Products

#### Using `groupby`

In [13]:
# Show the number products sold for region, state, and city.
# Rename the sum to "Total_Products_Sold".
groupby_rsc_df = as_years_df.groupby(['region','state','city']).sum('units_sold').reset_index().rename(columns={'units_sold':'Total_Products_Sold'})
# print (f"# of products sold by region, state and city\n{groupby_rsc_df[['region','state','city','Total_Products_Sold']].head(5).to_string(index=False)}\n")

# Show the top 5 results.
sort_groupby_rsc_df = groupby_rsc_df.sort_values(by=['Total_Products_Sold'], ascending=False, ignore_index=(True))
print(f"Top 5 # of products sold by region, state and city \n{sort_groupby_rsc_df[['region','state','city','Total_Products_Sold']].head(5).to_string(index=False)}")

Top 5 # of products sold by region, state and city 
   region      state          city  Total_Products_Sold
Northeast   New York      New York               111954
    South      Texas       Houston                90322
     West California San Francisco                85478
     West California   Los Angeles                76384
Southeast    Florida         Miami                73135


#### Using `pivot_table`

In [14]:
# Show the number products sold for region, state, and city.
# Rename the "units_sold" column to "Total_Products_Sold"
pt_rsc_df = as_years_df.pivot_table(index=['region','state','city'],
                                    values='units_sold',
                                    aggfunc='sum').reset_index().rename(columns={'units_sold':'Total_Products_Sold'})                                    
# print (f"# of products sold by region, state and city\n{pt_rsc_df.head(5).to_string(index=False)}\n")
# Show the top 5 results.
print(f"Top 5 # of products sold by region, state and city \n{pt_rsc_df.sort_values(by=['Total_Products_Sold'], ascending=False, ignore_index=(True)).head(5).to_string(index=False)}")

Top 5 # of products sold by region, state and city 
   region      state          city  Total_Products_Sold
Northeast   New York      New York               111954
    South      Texas       Houston                90322
     West California San Francisco                85478
     West California   Los Angeles                76384
Southeast    Florida         Miami                73135


### 3. Determine which Region had the Most Sales

#### Using `groupby`

In [15]:
# Show the total sales for the products sold for each region, state, and city.
# Rename the "total_sales" column to "Total Sales"
groupby_rsc_df = as_years_df.groupby(['region','state','city']).sum('total_sales').reset_index().rename(columns={'total_sales':'Total Sales'})
# print (f"sales  by region, state and city\n{groupby_rsc_df[['region','state','city','Total Sales']].head(5).to_string(index=False)}\n")
# Show the top 5 results.
sort_groupby_rsc_df = groupby_rsc_df.sort_values(by=['Total Sales'], ascending=False, ignore_index=(True))
print (f"top 5 sales  by region, state and city\n{sort_groupby_rsc_df[['region','state','city','Total Sales']].head(5).to_string(index=False)}")


top 5 sales  by region, state and city
   region          state          city  Total Sales
Northeast       New York      New York     39801235
     West     California San Francisco     33973228
Southeast        Florida         Miami     31600863
Southeast South Carolina    Charleston     29285637
Southeast        Florida       Orlando     27682851


#### Using `pivot_table`

In [16]:
# Show the total sales for the products sold for each region, state, and city.
# Optional: Rename the "total_sales" column to "Total Sales"
pt_rsc_df = as_years_df.pivot_table(index=['region','state','city'],
                                    values='total_sales',
                                    aggfunc='sum').reset_index().rename(columns={'total_sales':'Total Sales'})\
                                    .sort_values(by=['Total Sales'], ascending=False, ignore_index=(True))

# Show the top 5 results.
print (f"top 5 sales by region, state and city\n{pt_rsc_df[['region','state','city','Total Sales']].head(5).to_string(index=False)}\n")

top 5 sales by region, state and city
   region          state          city  Total Sales
Northeast       New York      New York     39801235
     West     California San Francisco     33973228
Southeast        Florida         Miami     31600863
Southeast South Carolina    Charleston     29285637
Southeast        Florida       Orlando     27682851



### 4. Determine which Retailer had the Most Sales

#### Using `groupby`

In [17]:
# Show the total sales for the products sold for each retailer, region, state, and city.
# Rename the "total_sales" column to "Total Sales"

groupby_rsc_df = as_years_df.groupby(['retailer','region','state','city']).sum('total_sales').reset_index().rename(columns={'total_sales':'Total Sales'})
# print (f"sales by retailer, region, state and city\n{groupby_rsc_df[['retailer','region','state','city','Total Sales']].head(5).to_string(index=False)}\n")

# Show the top 5 results.
sort_groupby_rsc_df = groupby_rsc_df.sort_values(by=['Total Sales'], ascending=False, ignore_index=(True))
print (f"top 5 sales by retailer, region, state and city\n{sort_groupby_rsc_df[['retailer','region','state','city','Total Sales']].head(5).to_string(index=False)}")


top 5 sales by retailer, region, state and city
   retailer    region          state          city  Total Sales
  West Gear      West     California San Francisco     32794405
     Kohl's      West     California   Los Angeles     25127160
Foot Locker Northeast       New York      New York     25008568
  West Gear      West     Washington       Seattle     24862675
Foot Locker Southeast South Carolina    Charleston     24822280


#### Using `pivot_table`

In [18]:
# Show the total sales for the products sold for each retailer, region, state, and city.
# Optional: Rename the "total_sales" column to "Total Sales"
pt_rsc_df = as_years_df.pivot_table(index=['retailer','region','state','city'],
                                    values='total_sales',
                                    aggfunc='sum').reset_index().rename(columns={'total_sales':'Total Sales'})\
                                    .sort_values(by=['Total Sales'], ascending=False, ignore_index=(True))

# Show the top 5 results.
print (f"top 5 sales by retailer, region, state and city\n{pt_rsc_df[['retailer','region','state','city','Total Sales']].head(5).to_string(index=False)}")


top 5 sales by retailer, region, state and city
   retailer    region          state          city  Total Sales
  West Gear      West     California San Francisco     32794405
     Kohl's      West     California   Los Angeles     25127160
Foot Locker Northeast       New York      New York     25008568
  West Gear      West     Washington       Seattle     24862675
Foot Locker Southeast South Carolina    Charleston     24822280


### 5. Determine which Retailer Sold the Most Women's Athletic Footwear

In [19]:
# Filter the sales data to get the women's athletic footwear sales data.
# init for a product selection
idx=4
products=as_years_df["product"].unique()
''' uncomment this section to enable product selection
# get column value for womens footwear
# print each unique product with the product Index
for index, value in enumerate(products):
    print(f"{index} : {value}")
idx=int(input('enter the number for the product : '))
'''
# filter product by womens footwear
# filtered_df = df[df['column_name'].isin(['value1', 'value2', 'value3'])]
as_wfw_df = as_years_df[as_years_df["product"].isin([products[idx]])]
print (f"\n{as_wfw_df[['retailer','region','state','city','product']].tail(3).to_string(index=False)}")


   retailer    region        state         city                   product
     Amazon Northeast        Maine     Portland Women's Athletic Footwear
     Amazon Northeast        Maine     Portland Women's Athletic Footwear
Foot Locker Northeast Pennsylvania Philadelphia Women's Athletic Footwear


### Using `groupby`

In [20]:
# Show the total number of women's athletic footwear sold for each retailer, region, state, and city.
# Rename the "units_sold" column to "{Product[idx]}_Units_Sold"
product_unit_sold = str(products[idx])+' Units Sold'
groupby_rsc_df = as_wfw_df.groupby(['retailer','region','state','city']).sum('units_sold').reset_index().rename(columns={'units_sold':product_unit_sold})
# print (f"{products[idx]}\n{groupby_rsc_df[['retailer','region','state','city',product_unit_sold]].head(5).to_string(index=False)}\n")

# Show the top 5 results.
sort_groupby_rsc_df = groupby_rsc_df.sort_values(by=[product_unit_sold], ascending=False, ignore_index=(True))
print(f"Sorted by {product_unit_sold}\n{sort_groupby_rsc_df[['retailer','region','state','city',product_unit_sold]].head(5).to_string(index=False)}")

Sorted by Women's Athletic Footwear Units Sold
     retailer    region          state          city  Women's Athletic Footwear Units Sold
    West Gear      West     California San Francisco                                 12107
  Foot Locker Northeast       New York      New York                                 10996
       Kohl's      West     California   Los Angeles                                 10826
  Foot Locker Southeast South Carolina    Charleston                                  8814
Sports Direct     South          Texas        Dallas                                  8790


#### Using `pivot_table`

In [21]:
# Show the total number of women's athletic footwear sold for each retailer, region, state, and city.
# Rename the "units_sold" column to "Womens_Footwear_Units_Sold"
product_unit_sold = str(products[idx])+' Units Sold'
pt_rsc_df = as_wfw_df.pivot_table(index=['retailer','region','state','city'],
                                    values='units_sold',
                                    aggfunc='sum').reset_index().rename(columns={'units_sold':product_unit_sold})
# print(f"\n{pt_rsc_df.head(5).to_string(index=False)}\n")
# Show the top 5 results.
print(f"Sorted by {product_unit_sold}\n{pt_rsc_df.sort_values(by=[product_unit_sold], ascending=False, ignore_index=(True)).head(5).to_string(index=False)}")

Sorted by Women's Athletic Footwear Units Sold
     retailer    region          state          city  Women's Athletic Footwear Units Sold
    West Gear      West     California San Francisco                                 12107
  Foot Locker Northeast       New York      New York                                 10996
       Kohl's      West     California   Los Angeles                                 10826
  Foot Locker Southeast South Carolina    Charleston                                  8814
Sports Direct     South          Texas        Dallas                                  8790


### 5. Determine the Day with the Most Women's Athletic Footwear Sales

In [22]:
# Create a pivot table with the 'invoice_date' column is the index, and the "total_sales" as the values.
pt_wfw_sales_df = as_wfw_df.pivot_table(index='invoice_date',
                                    values='total_sales',
                                    aggfunc='sum')\
                                    .rename(columns={'total_sales':'Total Sales'})
# Show the table.
pt_wfw_sales_df.head(5)

,Total Sales
invoice_date,
2020-01-04,402743
2020-01-05,141801
2020-01-11,129556
2020-01-17,173013
2020-01-22,388250


In [23]:
# Resample the pivot table into daily bins, and get the total sales for each day.
bin_daily_sum = pt_wfw_sales_df.resample('D').sum()
# Sort the resampled pivot table in ascending order on "Total Sales".
daily_high_wafs = bin_daily_sum.sort_values(by=['Total Sales'] , ascending=False).reset_index()
print   (f"The day with the Most {products[idx]} Sales is \n{daily_high_wafs.iloc[0,0].strftime('%Y-%m-%d')} with sales of {daily_high_wafs.iloc[0,1]}")

The day with the Most Women's Athletic Footwear Sales is 
2021-07-16 with sales of 1521825


### 6.  Determine the Week with the Most Women's Athletic Footwear Sales

In [24]:
# Resample the pivot table into weekly bins, and get the total sales for each week.
bin_weekly_sum = pt_wfw_sales_df.resample('1W').sum()

# Sort the resampled pivot table in ascending order on "Total Sales".
weekly_high_wafs = bin_weekly_sum.sort_values(by=['Total Sales'] , ascending=False).reset_index()
                
print (f"The week with the Most {products[idx]} Sales is \nweek {weekly_high_wafs.iloc[0, 0].week} of {weekly_high_wafs.iloc[0, 0].year} with sales of {weekly_high_wafs.iloc[0, 1]}")

The week with the Most Women's Athletic Footwear Sales is 
week 50 of 2021 with sales of 3098970
